<a href="https://colab.research.google.com/github/DEvauthentic/DEvauthentic/blob/main/SYSTEME_RAG_COMPLETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Installation de packages requis pour la construction du RAG
! pip install contextual-client

In [25]:
!pip install matplotlib tqdm requests

**Importation des différents modules**

In [26]:
import os
import json
import requests
from pathlib import Path
from typing import List, Optional, Dict
from IPython.display import display, JSON
import pandas as pd
from contextual import ContextualAI
import ast

S'authentifier au près de Contextual AI pour avoir une clé d'API

Ajouter la clé api de context ai à la variable d'environnement

In [27]:
!echo "CONTEXTUAL_API_KEY=" .env

CONTEXTUAL_API_KEY= .env


**Maintenant que l'environnement est initilisé avec la clé de l'api de context Ai, créons l'instance de ce dernier pret à etre utiliser**

In [39]:
from dotenv import load_dotenv
import os
load_dotenv()

# Initialize with your API key
client = ContextualAI(
    api_key="key-p_eu-gDkx-8DqFYklHUvSyCY0ETZFfrbwN0RaABKDzazzLVvI"
)

##Créons de la base de donnée des documents de finance pour avec Context Ai


In [40]:
datastore_name = 'Financial_Demo_RAG'

# Vérifier si le datastore existe
datastores = client.datastores.list()
existing_datastore = next((ds for ds in datastores if ds.name == datastore_name), None)

if existing_datastore:
    datastore_id = existing_datastore.id
    print(f"La base de connaissance existe déja et sera utilisée avec l'ID: {datastore_id}")
else:
    result = client.datastores.create(name=datastore_name)
    datastore_id = result.id
    print(f"Creation de nouvelle base de connaissance avec l'ID: {datastore_id}")

La base de connaissance existe déja et sera utilisée avec l'ID: 60c3ed0f-8499-4b95-b59f-ce97bf4eaa31


**Passons à l'étape de la création des chunks, les chunks sont les portions des documents qui seront ensuite transformés en embeddings qui à leur tour seront stockés dans une base de donnée Vectorielle, ici la base de donnée de Context Ai**

**Les formats de fichiers supportés :**

1.   PDF

2.  HTML
3.  DOC/DOCX


4.  PPT/PPTX



In [42]:
import os
import requests

# Créer un répertoire de données s'il n'existe pas
if not os.path.exists('data'):
    os.makedirs('data')

# Liste des fichiers avec les URL GitHub correspondantes
files_to_upload = [
    # Revenus trimestriels de NVIDIA 24/25
    ("A_Rev_by_Mkt_Qtrly_Trend_Q425.pdf", "https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/08-ai-workshop/data/A_Rev_by_Mkt_Qtrly_Trend_Q425.pdf"),
    # Revenus trimestriels de NVIDIA 22/23
    ("B_Q423-Qtrly-Revenue-by-Market-slide.pdf", "https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/08-ai-workshop/data/B_Q423-Qtrly-Revenue-by-Market-slide.pdf"),
    # Rapport de corrélations fallacieuses - exemple amusant de graphiques et d'analyse statistique
    ("C_Neptune.pdf", "https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/08-ai-workshop/data/C_Neptune.pdf"),
    # Un autre rapport de corrélations fallacieuses - exemple amusant de graphiques et d'analyse statistique
    ("D_Unilever.pdf", "https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/08-ai-workshop/data/D_Unilever.pdf")
]

**Télecharger les documents et les ajouter à la base de connaissance sur Context Ai ( Context Ai sura créer les chunks =) embeddings avant de les stocker ) dans la base de donnée**

In [43]:
# Télécharger et ingérer tous les fichiers
document_ids = []
for filename, url in files_to_upload:
    file_path = f'data/{filename}'

    # Télécharger le fichier s'il n'existe pas
    if not os.path.exists(file_path):
        print(f"Fetching {file_path}")
        try:
            response = requests.get(url)
            response.raise_for_status()  # Lève une exception pour les mauvais codes de statut
            with open(file_path, 'wb') as f:
                f.write(response.content)
        except Exception as e:
            print(f"Erreur lors du téléchargement de {filename}: {str(e)}")
            continue

    # Télécharger vers le datastore
    try:
        with open(file_path, 'rb') as f:
            ingestion_result = client.datastores.documents.ingest(datastore_id, file=f)
            document_id = ingestion_result.id
            document_ids.append(document_id)
            print(f"Fichier {filename} téléchargé avec succès vers le datastore {datastore_id}")
    except Exception as e:
        print(f"Erreur lors du téléchargement de {filename}: {str(e)}")

print(f"Fichiers téléchargés avec succès vers le datastore : {len(document_ids)}")
print(f"IDs des documents : {document_ids}")

Fetching data/A_Rev_by_Mkt_Qtrly_Trend_Q425.pdf
Fichier A_Rev_by_Mkt_Qtrly_Trend_Q425.pdf téléchargé avec succès vers le datastore 60c3ed0f-8499-4b95-b59f-ce97bf4eaa31
Fetching data/B_Q423-Qtrly-Revenue-by-Market-slide.pdf
Fichier B_Q423-Qtrly-Revenue-by-Market-slide.pdf téléchargé avec succès vers le datastore 60c3ed0f-8499-4b95-b59f-ce97bf4eaa31
Fetching data/C_Neptune.pdf
Fichier C_Neptune.pdf téléchargé avec succès vers le datastore 60c3ed0f-8499-4b95-b59f-ce97bf4eaa31
Fetching data/D_Unilever.pdf
Fichier D_Unilever.pdf téléchargé avec succès vers le datastore 60c3ed0f-8499-4b95-b59f-ce97bf4eaa31
Fichiers téléchargés avec succès vers le datastore : 4
IDs des documents : ['7a592fdd-e6ea-4a68-bc51-7909eaf8b3dd', '5e253926-e041-4857-bf35-8a94f88cc7a1', 'cdcd35b5-9753-4929-865f-1cd4ecc3f23e', 'deaa3c1b-f6d7-4cac-96aa-97a4f920bb63']


**En navigant sur le tableau de board de Context Ai dans la base de connaissance on voit que nos documents ont été bien ingérés**

***Vérifons ici cela***

In [44]:
metadata = client.datastores.documents.metadata(datastore_id = datastore_id, document_id = document_ids[0])
print("Document metadata:", metadata)

Document metadata: DocumentMetadata(id='7a592fdd-e6ea-4a68-bc51-7909eaf8b3dd', created_at='2025-09-04T09:38:03.612748', name='A_Rev_by_Mkt_Qtrly_Trend_Q425.pdf', status='completed', custom_metadata={}, custom_metadata_config={}, has_access=True, ingestion_config={'parsing': {'figure_captioning_prompt': None, 'figure_caption_mode': 'default', 'enable_split_tables': True, 'max_split_table_cells': 100, 'ocr_level': 'auto', 'use_hyperlink_extraction': False, 'enable_vlm_hierarchy_inference': True, 'layout_model': 'dit', 'vlm_captioning_model': 'gemini-2.0-flash-001', 'vlm_hierarchy_model': 'gemini-2.0-flash-001', 'vlm_doc_name_model': 'gemini-2.0-flash-001', 'vlm_markdown_reviser_model': None}, 'chunking': {'chunking_mode': 'hierarchy_depth', 'max_chunk_length_tokens': 768, 'min_chunk_length_tokens': 384, 'enable_hierarchy_based_contextualization': True, 'enable_contextualization': None, 'enable_section_based_contextualization': None}, 'html_config': {'max_recursive_depth': 5, 'markdown_li

Création de l'Agent LLM

*   Permet de prendre le contexte de la réponse prise par le retriver et la demande de l'utilisateur pour bien etre précis dans la réponse finale



*   On a le choix netre un grand nombre de LLM (Prenium, Frenuim partiel, Local) :
*   ChatGpt ( Prenium Très rapide, populaire )


*   Gemini ( Prenium, Performence reconnue , rapide)
*   Grok Ai ( Gratuit à certains niveau, performant )


*   DeepSeek
*   LLAMA











In [45]:
system_prompt = '''
You are a helpful AI assistant created by Contextual AI to answer questions about relevant documentation provided to you. Your responses should be precise, accurate, and sourced exclusively from the provided information. Please follow these guidelines:
* Only use information from the provided documentation. Avoid opinions, speculation, or assumptions.
* Use the exact terminology and descriptions found in the provided content.
* Keep answers concise and relevant to the user's question.
* Use acronyms and abbreviations exactly as they appear in the documentation or query.
* Apply markdown if your response includes lists, tables, or code.
* Directly answer the question, then STOP. Avoid additional explanations unless specifically relevant.
* If the information is irrelevant, simply respond that you don't have relevant documentation and do not provide additional comments or suggestions. Ignore anything that cannot be used to directly answer this query.
'''
agent_name = "Demo"

# Get list of existing agents
# Obtenir la liste des agents existants
agents = client.agents.list()

# Check if agent already exists
# Vérifier si l'agent existe déjà
existing_agent = next((agent for agent in agents if agent.name == agent_name), None)

if existing_agent:
    agent_id = existing_agent.id
    print(f"Using existing agent with ID: {agent_id}")
    # Utilisation de l'agent existant avec l'ID : {agent_id}
else:
    print("Creating new agent")
    # Création d'un nouvel agent
    app_response = client.agents.create(
        name=agent_name,
        description="Helpful Grounded AI Assistant",
        # Assistant IA utile basé sur les documents
        datastore_ids=[datastore_id],
        agent_configs={
        "global_config": {
            "enable_multi_turn": False # Turning this off for deterministic responses for this demo
            # Désactiver ceci pour des réponses déterministes pour cette démo
        }
        },
        suggested_queries=[
            "What was NVIDIA's annual revenue by fiscal year 2022 to 2025?",
            "When did NVIDIA's data center revenue overtake gaming revenue?",
            "What's the correlation between the distance between Neptune and the Sun and Burglary rates in the US?",
            "What's the correlation between Global revenue generated by Unilever Group and Google searches for 'lost my wallet'?",
            "Does this imply that Unilever Group's revenue is derived from lost wallets?",
            "What's the correlation between the distance between Neptune and the Sun and Global revenue generated by Unilever Group?"
        ]
    )
    agent_id = app_response.id
    print(f"Agent ID created: {agent_id}")
    # ID de l'agent créé : {agent_id}

Creating new agent
Agent ID created: 44e42ece-0963-4c5f-9da3-7649d0ef3969


**Testons l'agent en lui posant une question**

In [51]:
query_result = client.agents.query.create(
    agent_id=agent_id,
    messages=[{
        "content": "What was NVIDIA's Data Center revenue in Q4 FY25??",
        "role": "user"
    }]
)
print(query_result.message.content)

According to the provided NVIDIA quarterly revenue documentation, I can directly answer your question about Data Center revenue in Q4 FY25:

For Q4 FY25, NVIDIA's Data Center revenue was $35,580 million.[1]()()

For context, this figure represents the highest quarterly Data Center revenue shown in the provided dataset.

It marks an increase from $30,771 million in Q3 FY25 and $18,404 million in Q4 FY24.[1]()()


**Super !! Notre agent est bien opérationnel et répond de manière fluide au questions.**

**On passe à la dernière étape qui consite à tester avec des métrics bien précis pour s'assurer de la fiabilité de notre système RAG avant la mise en production du système**



*   *Accuracy (Exatitude des réponses de l'agent), oui l'agent répond mais est ce avec des réponses exactes ?* , posez une question par exemple : Quel était le CA de NVIDAI en 2023 (Ici on s'attend à une réponse exacte)

*   

*   Causation (Causabilité), oui c'est important, supposons que dans une réponse l'agent mentionne la chutte du CA de Nvidia était du au terrosrisme dans le sahel, la c'est vrai que Nvidia à connue une chutte de son CA mais la cause est loin d'etre nle terrorisme au sahel

*   Synthesis (Synthèse) : Ici il sagit de mesurer la capacité de l'agent à faire sortir une synthèse clair, par exemple : Quelle est la moyenne du CA de Nvidia de 2022 à 2025 ?, la belle réponse serait +47.3% en moyenne annuelle et non un nombre fixe


*   Limitations (Limites / Transparence) : Ce métric est très important car permet de voir à quel point l'agent allucine ou non, un agent normal devra savoir dire qu'il ne connais si réellement il ne connais pas au lieu de vouloir inventer ou de prendre un context non cohérent pour dévelloper, exemple : Que envisage de faire NVIDIA dans les 10 prochaines années ?, ici l'agent devra dire gentillement qu'il n'a pas de réponse au lieu de se mettre à la place de NVIDIA pour donner des réponses ni tete, ni queue


*   Evidence (Preuves / Citations) : Ici on teste pour voir l'agent fournit vraiment des sources au questions qu'on lui pose, pas juste entrain d'affirmer sans les sources.



*   Relevance (Pertinence) : Ici la pertinence est crutial car on a bien sur besoin que l'agent soit bien précis, pas la peine de fournir des réponses trop longues sans dire de bon, une réponse raisonnable et precis.






**Mettons cela en pratique pour voir**

***`LMUnit est l'outils ou le framework utimine pour mener cette mesure des métrics de notre Système RAG`***

In [47]:
#Préparons les différentes questions pour le fameux test (6 question, une question pour chaque métric)
unit_tests = [
      "Does the response accurately extract specific numerical data from the documents?",
      "Does the agent properly distinguish between correlation and causation?",
      "Are multi-document comparisons performed correctly with accurate calculations?",
      "Are potential limitations or uncertainties in the data clearly acknowledged?",
      "Are quantitative claims properly supported with specific evidence from the source documents?",
      "Does the response avoid unnecessary information?"
]

In [49]:
#Testons avec LMUNIT
response = client.lmunit.create(
                    query="What was NVIDIA's Data Center revenue in Q4 FY25?",
                   response = """NVIDIA's Data Center revenue for Q4 FY25 was $35,580 million.[1]()

                                This represents a significant increase from the previous quarter (Q3 FY25) when Data Center revenue was $30,771 million.[1]()

                                The full quarterly trend for Data Center revenue in FY25 was:
                                - Q4 FY25: $35,580 million
                                - Q3 FY25: $30,771 million
                                - Q2 FY25: $26,272 million
                                - Q1 FY25: $22,563 million.[1]()
                              """,
                    unit_test="Does the response avoid unnecessary information?"
                )
print(response)

LMUnitCreateResponse(score=2.309)


In [53]:
# ❌ Mauvaise pratique : tu donnes une réponse trop longue comme input
# ✅ Bonne pratique : tu testes la réponse réelle du modèle

# Supposons que tu aies un agent
actual_response = "According to the provided NVIDIA quarterly revenue documentation, I can directly answer your question about Data Center revenue in Q4 FY25:\n\nFor Q4 FY25, NVIDIA's Data Center revenue was $35,580 million.[1]()()\n\nFor context, this figure represents the highest quarterly Data Center revenue shown in the provided dataset.\n\nIt marks an increase from $30,771 million in Q3 FY25 and $18,404 million in Q4 FY24.[1]()()"

# Maintenant, tu évalues cette réponse
evaluation = client.lmunit.create(
    query="What was NVIDIA's Data Center revenue in Q4 FY25?",
    response=actual_response,
    unit_test="Does the response avoid unnecessary information?"
)

print(evaluation.score)  # Afficher le score au lieu de 'passed'

3.104
